<a href="https://colab.research.google.com/github/veiro/tesis-imputacion-datos/blob/main/codigo/imputadores/GAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo:
Probar GAIN para imputar datos en el juego de datos de Covid.

# Trabajo en siguiente iteracion
La salida de este notebook es un set de datos con datos imputados llamado dataset-GAIN.

# Configuracion ambiente

In [ ]:
USAR_GOOGLE_DRIVE =  True
MODO_DESARROLLO = False

In [ ]:
import os
import sys

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import pandas
from sklearn.model_selection import train_test_split
from google.colab import drive

from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
if (USAR_GOOGLE_DRIVE):
  drive.mount('/content/gdrive')
  PATH_DATA_PROCESADA = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/data/data-procesada"
  PATH_IMPUTATION = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/imputacion/GAIN/"
  sys.path.append('/content/gdrive/MyDrive/Maestria/Tesis/Codigo/utils')
  import utils as utils

else:
  PATH_DATA_PROCESADA = "data/data-procesada"

import importlib
importlib.reload(utils)

if (25 ==  utils.version()):
  print("version correcta")
else:
  raise Exception("Version de Util vieja")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
version correcta


## Dependencias de GAIN

In [ ]:
ruta_req= PATH_IMPUTATION+'requirements.txt'
!pip install -r $ruta_req &> /dev/null

In [ ]:
!python3 -m pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.4.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.5
cuda-python==12.2.1
cudf-cu12 @ https:

# Preprocesamiento datos

## Carga datasets

In [ ]:
# para hacer pruebas bajo la cantidad


df_train_encoded, df_train_encoded_missing, mask_train_open = utils.obtenerDatosTrain(MODO_DESARROLLO, PATH_DATA_PROCESADA)


# esto lo dejo asi por que se usa pila :(
data_transformed = df_train_encoded


# GAIN


In [ ]:
# Necessary packages
import numpy as np
from keras.datasets import mnist
import numpy as np
#import tensorflow as tf
##IF USING TF 2 use following import to still use TF < 2.0 Functionalities
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
from tqdm import tqdm


Instructions for updating:
non-resource variables are not supported in the long term


## utils


In [ ]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.

  Args:
    - data: original data

  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)

    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i].astype(float)
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}

  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters


def renormalization (norm_data, norm_parameters):
  '''Renormalize data from [0, 1] range to the original range.

  Args:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization

  Returns:
    - renorm_data: renormalized original data
  '''

  min_val = norm_parameters['min_val']
  max_val = norm_parameters['max_val']

  _, dim = norm_data.shape
  renorm_data = norm_data.copy()

  for i in range(dim):
    renorm_data[:,i] = renorm_data[:,i] * (max_val[i] + 1e-6)
    renorm_data[:,i] = renorm_data[:,i] + min_val[i]

  return renorm_data


def rounding (imputed_data, data_x):
  '''Round imputed data for categorical variables.

  Args:
    - imputed_data: imputed data
    - data_x: original data with missing values

  Returns:
    - rounded_data: rounded imputed data
  '''

  _, dim = data_x.shape
  rounded_data = imputed_data.copy()

  for i in range(dim):
    temp = data_x[~np.isnan(data_x[:, i]), i]
    # Only for the categorical variable
    if len(np.unique(temp)) < 20:
      rounded_data[:, i] = np.round(rounded_data[:, i])

  return rounded_data



def xavier_init(size):
  '''Xavier initialization.

  Args:
    - size: vector size

  Returns:
    - initialized random vector.
  '''
  in_dim = size[0]
  xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
  return tf.random_normal(shape = size, stddev = xavier_stddev)


def binary_sampler(p, rows, cols):
  '''Sample binary random variables.

  Args:
    - p: probability of 1
    - rows: the number of rows
    - cols: the number of columns

  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = [rows, cols])
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix


def uniform_sampler(low, high, rows, cols):
  '''Sample uniform random variables.

  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns

  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = [rows, cols])


def sample_batch_index(total, batch_size):
  '''Sample index of the mini-batch.

  Args:
    - total: total number of samples
    - batch_size: batch size

  Returns:
    - batch_idx: batch index
  '''
  total_idx = np.random.permutation(total)
  batch_idx = total_idx[:batch_size]
  return batch_idx

## data loader

In [ ]:
def data_loader (data_name, miss_rate):
  '''Loads datasets and introduce missingness.

  Args:
    - data_name: letter, spam, or mnist
    - miss_rate: the probability of missing components

  Returns:
    data_x: original data
    miss_data_x: data with missing values
    data_m: indicator matrix for missing components
  '''
  # Load data
  if data_name in ['letter', 'spam']:
    file_name = PATH_IMPUTATION + '/data/'+data_name+'.csv'
    data_x = np.loadtxt(file_name, delimiter=",", skiprows=1)
  elif data_name == 'mnist':
    (data_x, _), _ = mnist.load_data()
    data_x = np.reshape(np.asarray(data_x), [60000, 28*28]).astype(float)
  elif data_name == 'covid':
    print("Proceso covid dataset")
    # no uso los argumentos ni nada, por ahora solo carga train
    # para hacer pruebas bajo la cantidad
    data_x = df_train_encoded.to_numpy()
  #np.loadtxt(file_name, delimiter=",", skiprows=1)
  # Parameters
  # no, dim = data_x.shape

  # Introduce missing data
  #data_m = binary_sampler(1-miss_rate, no, dim)
  #miss_data_x = data_x.copy()
  #miss_data_x[data_m == 0] = np.nan
  miss_data_x = df_train_encoded_missing.to_numpy()
  data_m= mask_train_open
  return data_x, miss_data_x, mask_train_open

## **GAIN**


In [ ]:
#tf.compat.v1.disable_eager_execution()
sess = tf.Session()
 ## GAIN functions
X = tf.placeholder(tf.float32, shape = [None, 34])
  # Mask vector
M = tf.placeholder(tf.float32, shape = [None, 34])


def gain (data_x, gain_parameters):
  global X
  global M
  global sess
  '''Impute missing values in data_x

  Args:
    - data_x: original data with missing values
    - gain_parameters: GAIN network parameters:
      - batch_size: Batch size
      - hint_rate: Hint rate
      - alpha: Hyperparameter
      - iterations: Iterations

  Returns:
    - imputed_data: imputed data
  '''
  data_m = 1-np.isnan(data_x)

  # System parameters
  batch_size = gain_parameters['batch_size']
  hint_rate = gain_parameters['hint_rate']
  alpha = gain_parameters['alpha']
  iterations = gain_parameters['iterations']

  # Other parameters
  no, dim = data_x.shape

  # Hidden state dimensions
  h_dim = int(dim)

  # Normalization
  norm_data, norm_parameters = normalization(data_x)
  norm_data_x = np.nan_to_num(norm_data, 0)

  ## GAIN architecture
  # Input placeholders
  # Data vector
  #X = tf.placeholder(tf.float32, shape = [None, dim])
  # Mask vector
  #M = tf.placeholder(tf.float32, shape = [None, dim])
  # Hint vector
  H = tf.placeholder(tf.float32, shape = [None, dim])

  # Discriminator variables
  D_W1 = tf.Variable(xavier_init([dim*2, h_dim])) # Data + Hint as inputs
  D_b1 = tf.Variable(tf.zeros(shape = [h_dim]))

  D_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
  D_b2 = tf.Variable(tf.zeros(shape = [h_dim]))

  D_W3 = tf.Variable(xavier_init([h_dim, dim]))
  D_b3 = tf.Variable(tf.zeros(shape = [dim]))  # Multi-variate outputs

  theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]

  #Generator variables
  # Data + Mask as inputs (Random noise is in missing components)
  G_W1 = tf.Variable(xavier_init([dim*2, h_dim]))
  G_b1 = tf.Variable(tf.zeros(shape = [h_dim]))

  G_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
  G_b2 = tf.Variable(tf.zeros(shape = [h_dim]))

  G_W3 = tf.Variable(xavier_init([h_dim, dim]))
  G_b3 = tf.Variable(tf.zeros(shape = [dim]))

  theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]

  ## GAIN functions
  # Generator
  def generator(x,m):
    # Concatenate Mask and Data
    inputs = tf.concat(values = [x, m], axis = 1)
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)
    # MinMax normalized output
    G_prob = tf.nn.sigmoid(tf.matmul(G_h2, G_W3) + G_b3)
    return G_prob

  # Discriminator
  def discriminator(x, h):
    # Concatenate Data and Hint
    inputs = tf.concat(values = [x, h], axis = 1)
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
    D_logit = tf.matmul(D_h2, D_W3) + D_b3
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob

  ## GAIN structure
  # Generator
  G_sample = generator(X, M)

  # Combine with observed data
  Hat_X = X * M + G_sample * (1-M)

  # Discriminator
  D_prob = discriminator(Hat_X, H)

  ## GAIN loss
  D_loss_temp = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) \
                                + (1-M) * tf.log(1. - D_prob + 1e-8))

  G_loss_temp = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))

  MSE_loss = \
  tf.reduce_mean((M * X - M * G_sample)**2) / tf.reduce_mean(M)

  D_loss = D_loss_temp
  G_loss = G_loss_temp + alpha * MSE_loss

  ## GAIN solver
  D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
  G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

  ## Iterations
  #sess = tf.Session()
  sess.run(tf.global_variables_initializer())

  # Start Iterations
  for it in tqdm(range(iterations)):

    # Sample batch
    batch_idx = sample_batch_index(no, batch_size)
    X_mb = norm_data_x[batch_idx, :]
    M_mb = data_m[batch_idx, :]
    # Sample random vectors
    Z_mb = uniform_sampler(0, 0.01, batch_size, dim)
    # Sample hint vectors
    H_mb_temp = binary_sampler(hint_rate, batch_size, dim)
    H_mb = M_mb * H_mb_temp

    # Combine random vectors with observed vectors
    X_mb = M_mb * X_mb + (1-M_mb) * Z_mb

    _, D_loss_curr = sess.run([D_solver, D_loss_temp],
                              feed_dict = {M: M_mb, X: X_mb, H: H_mb})
    _, G_loss_curr, MSE_loss_curr = \
    sess.run([G_solver, G_loss_temp, MSE_loss],
             feed_dict = {X: X_mb, M: M_mb, H: H_mb})
    print("MSE_loss_curr: ", MSE_loss_curr)

  ## Return imputed data
  Z_mb = uniform_sampler(0, 0.01, no, dim)
  M_mb = data_m
  X_mb = norm_data_x
  X_mb = M_mb * X_mb + (1-M_mb) * Z_mb

  imputed_data = sess.run([G_sample], feed_dict = {X: X_mb, M: M_mb})[0]

  imputed_data = data_m * norm_data_x + (1-data_m) * imputed_data

  # Renormalization
  imputed_data = renormalization(imputed_data, norm_parameters)

  # Rounding
  imputed_data = rounding(imputed_data, data_x)

  saver = tf.compat.v1.train.Saver()
  saver.save(sess, PATH_IMPUTATION + '/modelo/GAIN_Desarrollo=' + str(MODO_DESARROLLO) + '.ckpt')

  return imputed_data, G_sample, saver

## main

In [ ]:
%%time
def main (data_name, miss_rate= 0.2, batch_size=32, hint_rate=0.9, alpha= 100,iterations=10000):
  data_name = data_name
  miss_rate = miss_rate

  gain_parameters = {'batch_size': batch_size,
                      'hint_rate': hint_rate,
                      'alpha': alpha,
                      'iterations': iterations}
  ori_data_x, miss_data_x, data_m = data_loader(data_name, miss_rate)
  imputed_data_x, G_sample,  saver = gain(miss_data_x, gain_parameters)
  rmse = utils.rmse(ori_data_x, imputed_data_x, data_m)
  return imputed_data_x, rmse, saver, G_sample

imputed_data_x, rmse, saver, G_sample= main(data_name='covid',miss_rate= 0.2, batch_size=32, hint_rate=0.9, alpha= 100,iterations=10000)
print('rmse', rmse)


Proceso covid dataset


  0%|          | 3/10000 [00:00<19:28,  8.56it/s]

MSE_loss_curr:  0.2774403
MSE_loss_curr:  0.25451776
MSE_loss_curr:  0.24702792


  0%|          | 5/10000 [00:00<16:12, 10.27it/s]

MSE_loss_curr:  0.23654245
MSE_loss_curr:  0.23515773
MSE_loss_curr:  0.23348258


...

100%|█████████▉| 9997/10000 [13:40<00:00, 12.15it/s]

MSE_loss_curr:  0.00036186917
MSE_loss_curr:  0.000116739844
MSE_loss_curr:  0.0013523349


100%|██████████| 10000/10000 [13:40<00:00, 12.19it/s]

MSE_loss_curr:  0.00015591917
MSE_loss_curr:  9.7563214e-05


rmse 18782.75954719253
CPU times: user 14min 26s, sys: 15.4 s, total: 14min 41s
Wall time: 14min 9s


# Prueba en test

## cargo los datos

In [ ]:
df_test_encoded, df_test_encoded_missing, mask_test_open = utils.obtenerDatosTest(MODO_DESARROLLO, PATH_DATA_PROCESADA)


# esto lo dejo asi por que se usa pila :(
new_data = df_test_encoded

In [ ]:
# Later, to restore the session and variables
#with tf.compat.v1.Session() as sess:
saver = tf.compat.v1.train.Saver()
saver.restore(sess, PATH_IMPUTATION + '/modelo/GAIN_Desarrollo=' + str(MODO_DESARROLLO) + '.ckpt')

no, dim = df_test_encoded_missing.shape

data_m = 1-pandas.isna(df_test_encoded_missing) # 1 si i no es Nan, 0 si es nan


#X = tf.placeholder(tf.float32, shape=[None, dim])
#M = tf.placeholder(tf.float32, shape=[None, dim])

norm_data, norm_parameters = normalization(df_test_encoded)



norm_data = norm_data.astype(np.float32)
norm_data = np.nan_to_num(norm_data, 0)

Z_mb = uniform_sampler(0, 0.01, no, dim)
M_mb = data_m
X_mb = norm_data
X_mb = M_mb * X_mb + (1-M_mb) * Z_mb

X_mb = X_mb.astype(np.float32)
# Continue with the rest of the code, such as making predictions on a new dataset
new_data_x = X_mb  # Replace this with your new dataset
new_data_m = M_mb  # Replace this with the corresponding missingness indicator matrix


imputed_new_data_x = sess.run([G_sample], feed_dict={X: new_data_x, M: new_data_m})[0]

imputed_new_data_x = data_m * norm_data + (1-data_m) * imputed_new_data_x
# Renormalization
imputed_data = renormalization(imputed_new_data_x, norm_parameters)

# Rounding
imputed_data = rounding(imputed_data, df_test_encoded_missing)

#data_m = data_m.astype(int)
rmse = utils.rmse(df_test_encoded, imputed_data, mask_test_open)
print('rmse', rmse)

imputed_data = pandas.DataFrame(imputed_data, columns = data_transformed.columns)
imputed_data.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_GAIN_encoded_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)




rmse 28625.115360338477


## Predecir

In [ ]:
imputed_data.shape

(897667, 34)

In [ ]:
%%time
imputed_data_decoded=utils.decode(imputed_data, data_transformed, PATH_DATA_PROCESADA)

897667it [02:17, 6508.42it/s]


CPU times: user 2min 18s, sys: 1.12 s, total: 2min 19s
Wall time: 2min 21s


In [ ]:

imputed_data_decoded.shape

(897667, 10)

In [ ]:
imputed_data_decoded.isna().sum()

Province_State         0
Country_Region         0
Last_Update            0
Lat                    0
Long_                  0
Confirmed              0
Deaths                 0
Incident_Rate          0
Case_Fatality_Ratio    0
date                   0
dtype: int64

In [ ]:
imputed_data_decoded['Country_Region'].value_counts().count()

25

In [ ]:
if (USAR_GOOGLE_DRIVE):
  imputed_data_decoded.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_GAIN._Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)